In [151]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import gkey

In [152]:
#read census csv file
census_df = pd.read_csv("censusdata.csv")
census_df.head()

,Zipcode,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate
0,601,17599.0,38.9,11757.0,7041.0,11282.0,64.105915
1,602,39209.0,40.9,16190.0,8978.0,20428.0,52.100283
2,603,50135.0,40.4,16645.0,10897.0,25176.0,50.216416
3,606,6304.0,42.8,13387.0,5960.0,4092.0,64.911168
4,610,27590.0,41.4,18741.0,9266.0,12553.0,45.498369


In [153]:
#read zip code csv file
zip_df = pd.read_csv("zip_code_database.csv")
# only utilize zip city state country lat lng
zip_df = zip_df.loc[:, ['zip','primary_city', 'state', 'country', 'latitude', 'longitude']]
zip_df = zip_df.rename(columns={"zip": "Zipcode"})
zip_df.head()

,Zipcode,primary_city,state,country,latitude,longitude
0,501,Holtsville,NY,US,40.81,-73.04
1,544,Holtsville,NY,US,40.81,-73.04
2,601,Adjuntas,PR,US,18.16,-66.72
3,602,Aguada,PR,US,18.38,-67.18
4,603,Aguadilla,PR,US,18.43,-67.15


In [171]:
# Merge Census and Zipcode dataframes using an inner join
merge_table = pd.merge(census_df, zip_df, on="Zipcode")
merge_table['location'] = list(zip(merge_table["latitude"] , merge_table["longitude"]))
# merge_table = merge_table['latitude'].astype(float)
# merge_table = merge_table['longitude'].astype(float)
merge_table.head()

,Zipcode,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,primary_city,state,country,latitude,longitude,location
0,601,17599.0,38.9,11757.0,7041.0,11282.0,64.105915,Adjuntas,PR,US,18.16,-66.72,"(18.16, -66.72)"
1,602,39209.0,40.9,16190.0,8978.0,20428.0,52.100283,Aguada,PR,US,18.38,-67.18,"(18.38, -67.18)"
2,603,50135.0,40.4,16645.0,10897.0,25176.0,50.216416,Aguadilla,PR,US,18.43,-67.15,"(18.43, -67.15)"
3,606,6304.0,42.8,13387.0,5960.0,4092.0,64.911168,Maricao,PR,US,18.18,-66.98,"(18.18, -66.98)"
4,610,27590.0,41.4,18741.0,9266.0,12553.0,45.498369,Anasco,PR,US,18.28,-67.14,"(18.28, -67.14)"


In [155]:
# pull in cell tower csv.

cell_df = pd.read_csv("OCID-diff-cell-export-2019-09-23-T000000.csv")

# limit mcc code to only view US cell towers. mcc 310 thru 316

cell_df = cell_df[(cell_df['mcc'] >= 310) & (cell_df['mcc'] <= 316)]

cell_df.head()

,radio,mcc,net,area,cell,unit,lon,lat,range,samples,changeable,created,updated,averageSignal
5130,LTE,310,410,26392,62427664,0,-89.945058,38.570057,6091,18,1,1372431182,1569163819,0
5358,UMTS,310,410,6992,131958701,0,-71.243112,42.705354,8239,8,1,1238903547,1569183960,0
5402,UMTS,310,410,30952,86998975,0,-91.458878,44.819621,3735,12,1,1317080006,1569193913,0
5403,UMTS,310,410,27398,231629347,0,-80.652073,28.030082,6537,78,1,1318201398,1569169421,0
5404,UMTS,310,410,27398,231629346,0,-80.623494,28.043499,3775,31,1,1318201500,1569170874,0


In [156]:
# round lat and longitude
cell_df = cell_df.round({'lon': 2, 'lat': 2})

cell_df['location'] = list(zip(cell_df["lat"] , cell_df["lon"]))

cell_df.head()

,radio,mcc,net,area,cell,unit,lon,lat,range,samples,changeable,created,updated,averageSignal,location
5130,LTE,310,410,26392,62427664,0,-89.95,38.57,6091,18,1,1372431182,1569163819,0,"(38.57, -89.95)"
5358,UMTS,310,410,6992,131958701,0,-71.24,42.71,8239,8,1,1238903547,1569183960,0,"(42.71, -71.24)"
5402,UMTS,310,410,30952,86998975,0,-91.46,44.82,3735,12,1,1317080006,1569193913,0,"(44.82, -91.46)"
5403,UMTS,310,410,27398,231629347,0,-80.65,28.03,6537,78,1,1318201398,1569169421,0,"(28.03, -80.65)"
5404,UMTS,310,410,27398,231629346,0,-80.62,28.04,3775,31,1,1318201500,1569170874,0,"(28.04, -80.62)"


In [157]:
df = cell_df[(cell_df['radio'] == "LTE")]
df.head()

,radio,mcc,net,area,cell,unit,lon,lat,range,samples,changeable,created,updated,averageSignal,location
5130,LTE,310,410,26392,62427664,0,-89.95,38.57,6091,18,1,1372431182,1569163819,0,"(38.57, -89.95)"
5409,LTE,310,410,12994,218632723,0,-78.81,35.76,1454,4,1,1326722189,1569121159,0,"(35.76, -78.81)"
5416,LTE,310,410,55508,56082592,0,-117.09,33.50,1405,2,1,1333304933,1569188822,0,"(33.5, -117.09)"
5443,LTE,310,410,56868,4994334,0,-121.48,38.44,1463,2,1,1354314460,1569162782,0,"(38.44, -121.48)"
5450,LTE,310,410,36111,171900943,0,-121.44,38.49,6926,29,1,1361407340,1569117472,0,"(38.49, -121.44)"


In [176]:
df = df.sort_values(["location"], ascending=False)
df.count

<bound method DataFrame.count of       radio  mcc  net   area       cell  unit  longitude  latitude  range  \
27185   LTE  310  410  37148  108877064     0    -117.81     48.63   1000   
15600   LTE  310  410  37148  108878344     0    -118.05     48.61  21804   
9972    LTE  310  410  37148  108878351     0    -118.06     48.61  21508   
5454    LTE  310  260  41397   27803297     0    -122.22     47.94   1000   
18188   LTE  310  260  11304   21657356     0    -122.23     47.91   1025   
21610   LTE  310  260  11334   21695766     0    -122.14     47.78   1091   
17468   LTE  310  260  11334   21693441     0    -122.20     47.73   1384   
24664   LTE  310  120  23313  190861842    -1    -122.29     47.72   1000   
24663   LTE  310  120  23313  190861829    -1    -122.29     47.72   1000   
24662   LTE  310  120  23313  190861832    -1    -122.29     47.72   1000   
29847   LTE  310  120  23313  190861838    -1    -122.29     47.72   1000   
14095   LTE  310  410  37123  108569602    

In [173]:
df = df.rename(columns={"lat": "latitude","lon": "longitude"})
merge_tower = pd.merge(df, merge_table, on=["location"])
merge_tower.head()

,radio,mcc,net,area,cell,unit,longitude_x,latitude_x,range,samples,...,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,primary_city,state,country,latitude_y,longitude_y
0,LTE,310,120,23313,190958342,-1,-122.30,47.71,1000,22,...,37.8,61014.0,39379.0,5499.0,13.476950,Seattle,WA,US,47.71,-122.30
1,LTE,310,120,23313,190861831,-1,-122.30,47.71,2307,21,...,37.8,61014.0,39379.0,5499.0,13.476950,Seattle,WA,US,47.71,-122.30
2,LTE,310,120,23313,190958348,-1,-122.30,47.71,1000,5,...,37.8,61014.0,39379.0,5499.0,13.476950,Seattle,WA,US,47.71,-122.30
3,LTE,310,120,23313,190861828,-1,-122.30,47.71,1988,14,...,37.8,61014.0,39379.0,5499.0,13.476950,Seattle,WA,US,47.71,-122.30
4,LTE,310,260,11316,20988427,0,-122.33,47.61,1000,2,...,38.9,68750.0,70914.0,2061.0,16.610251,Seattle,WA,US,47.61,-122.33


In [87]:
# merge cell_df and merge_table
data_df = pd.merge(merge_table, cell_df, on="location")
data_df.head()

,Zipcode,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,primary_city,state,country,...,cell,unit,lon,lat,range,samples,changeable,created,updated,averageSignal
0,771,37705.0,38.0,20332.0,10254.0,17258.0,45.771118,Las Piedras,PR,US,...,8853770,0,-65.86,18.18,1000,1,1,1569189163,1569189163,0
1,771,37705.0,38.0,20332.0,10254.0,17258.0,45.771118,Las Piedras,PR,US,...,8065286,0,-65.86,18.18,1000,1,1,1569189299,1569189299,0
2,771,37705.0,38.0,20332.0,10254.0,17258.0,45.771118,Las Piedras,PR,US,...,8062730,0,-65.86,18.18,1000,1,1,1569189381,1569189381,0
3,771,37705.0,38.0,20332.0,10254.0,17258.0,45.771118,Las Piedras,PR,US,...,8062723,0,-65.86,18.18,1000,1,1,1569189392,1569189392,0
4,777,40611.0,37.7,18684.0,9220.0,20541.0,50.579892,Juncos,PR,US,...,8085766,0,-65.91,18.22,1000,1,1,1569189533,1569189533,0


In [5]:
# # Target city
# target_city = "Boise, Idaho"

# # Build the endpoint URL
# target_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
#     'address={0}&key={1}').format(target_city, gkey)



In [6]:
# # Run a request to endpoint and convert result to json
# geo_data = requests.get(target_url).json()

# # Print the json (pretty printed)
# print(json.dumps(geo_data, indent=4, sort_keys=True))

{
    "results": [
        {
            "address_components": [
                {
                    "long_name": "Boise",
                    "short_name": "Boise",
                    "types": [
                        "locality",
                        "political"
                    ]
                },
                {
                    "long_name": "Ada County",
                    "short_name": "Ada County",
                    "types": [
                        "administrative_area_level_2",
                        "political"
                    ]
                },
                {
                    "long_name": "Idaho",
                    "short_name": "ID",
                    "types": [
                        "administrative_area_level_1",
                        "political"
                    ]
                },
                {
                    "long_name": "United States",
                    "short_name": "US",
                    "types": [
          